# Data Analysis Project: 5 - Pet Shelter Analysis

## Meets Expectations Requirements:

### 1. Is there an area where more pets are found?

### 2-a. What is the average number of pets found in a month in the year 2015?

### 2-b. Are there months where there is a higher number of animals found?

### 3. What is the ratio of incoming pets vs. adopted pets?

### 4. What is the distribution of the types of animals in the shelter?

### 5. What are the adoption rates for specific breeds?

### 6. What are the adoption rates for different colorings?

### 7. About how many animals are spayed/neutered each month?

## Exceeds Expectations Requirements:

### 1-a. How many animals in the shelter are repeats?

### 1-b. Which animal was returned to the shelter the most?

### 2. What are the adoption rates for the following age groups?
- baby: 4 months and less
- young: 5 months - 2 years
- adult: 3 years - 10 years
- senior: 11+

### 3. If spay/neuter for a dog costs \$100 and a spay/neuter for a cat costs \$50, how much did the shelter spend in 2015 on these procedures?